In [1]:
import osmnx as ox
import networkx as nx
import igraph as ig
import pandas as pd
import numpy as np

In [2]:
data1 = pd.read_csv('./data/day_data/day_1.csv')
l1 = len(data1)

In [3]:
data1 = data1[~(data1['new_duration'] == np.inf)]
data1.reset_index(inplace=True, drop=True)
l2 = len(data1)
print(f'原数据{l1}条, 去除不可达后数据{l2}条, 去除数据{l1-l2}条')

原数据308629条, 去除不可达后数据307790条, 去除数据839条


In [4]:
# %%time
# pd.to_datetime(data1.iloc[0, 0]) + pd.Timedelta(seconds=175)

In [5]:
data1['pickup_datetime'] = pd.to_datetime(data1['pickup_datetime'])

In [6]:
def get_timedelta(x):
    x = round(x)
    return pd.Timedelta(seconds=x)

In [7]:
%%time
data1['new_duration'] = data1['new_duration'].apply(get_timedelta)

Wall time: 4.25 s


In [8]:
data1['dropoff_datetime'] = data1['pickup_datetime'] + data1['new_duration']

In [9]:
data1

,pickup_datetime,dropoff_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,O,D,duration,weekday,day,hour,new_duration
0,2014-01-01 00:00:00,2014-01-01 00:02:55,-73.979904,40.761246,-73.973010,40.759007,1776,1724,120,2,1,0,0 days 00:02:55
1,2014-01-01 00:00:00,2014-01-01 00:02:18,-74.000610,40.727272,-73.993050,40.725500,816,766,120,2,1,0,0 days 00:02:18
2,2014-01-01 00:00:00,2014-01-01 00:04:19,-73.971300,40.792706,-73.959910,40.800747,2586,2796,180,2,1,0,0 days 00:04:19
3,2014-01-01 00:00:00,2014-01-01 00:02:02,-73.992710,40.743020,-73.989044,40.743366,1296,1312,180,2,1,0,0 days 00:02:02
4,2014-01-01 00:00:00,2014-01-01 00:01:30,-73.973420,40.751488,-73.973290,40.755287,1552,1635,240,2,1,0,0 days 00:01:30
...,...,...,...,...,...,...,...,...,...,...,...,...,...
307785,2014-01-01 23:59:53,2014-01-02 00:11:02,-73.954040,40.787520,-73.986720,40.764965,2462,1932,552,2,1,23,0 days 00:11:09
307786,2014-01-01 23:59:54,2014-01-02 00:07:32,-73.984210,40.760310,-73.999430,40.746100,1752,1315,287,2,1,23,0 days 00:07:38
307787,2014-01-01 23:59:54,2014-01-02 00:16:15,-74.006000,40.735764,-73.965904,40.766144,1095,1908,708,2,1,23,0 days 00:16:21
307788,2014-01-01 23:59:55,2014-01-02 00:06:56,-73.990425,40.730553,-73.988460,40.748577,922,1453,363,2,1,23,0 days 00:07:01


In [10]:
data1.to_csv('./data/day_data/use_day_1.csv', index=False)

## 以下省略

In [11]:
# 空驶时间阈值 阈值越大连接数越多，车辆逐渐减少，空驶率逐渐增大
# 此处需考虑司机在原地等待的时间
theta = 15
# 预知时长 时长逐渐增长，连接数逐渐增多多，车辆数逐渐减少， 愈加难以等待或者预测
pred_time = 30
# 延误时间阈值
delay = 0

In [15]:
# 构建图网络
G = ox.load_graphml('./data/Manhattan.graphml')
G_ig = ig.Graph(directed=True)
G_ig.add_vertices(list(G.nodes()))
G_ig.add_edges(list(G.edges()))
travel_time = pd.read_csv('./multi/travel_time.csv')
G_ig.es['travel_time'] = travel_time['travel_time']

In [16]:
data1['pred_time'] = data1['pickup_datetime'] + pd.Timedelta(minutes=pred_time)

In [23]:
data1

,pickup_datetime,dropoff_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,O,D,duration,weekday,day,hour,new_duration,pred_time
0,2014-01-01 00:00:00,2014-01-01 00:02:55,-73.979904,40.761246,-73.973010,40.759007,1776,1724,120,2,1,0,0 days 00:02:55,2014-01-01 00:30:00
1,2014-01-01 00:00:00,2014-01-01 00:02:18,-74.000610,40.727272,-73.993050,40.725500,816,766,120,2,1,0,0 days 00:02:18,2014-01-01 00:30:00
2,2014-01-01 00:00:00,2014-01-01 00:04:19,-73.971300,40.792706,-73.959910,40.800747,2586,2796,180,2,1,0,0 days 00:04:19,2014-01-01 00:30:00
3,2014-01-01 00:00:00,2014-01-01 00:02:02,-73.992710,40.743020,-73.989044,40.743366,1296,1312,180,2,1,0,0 days 00:02:02,2014-01-01 00:30:00
4,2014-01-01 00:00:00,2014-01-01 00:01:30,-73.973420,40.751488,-73.973290,40.755287,1552,1635,240,2,1,0,0 days 00:01:30,2014-01-01 00:30:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307785,2014-01-01 23:59:53,2014-01-02 00:11:02,-73.954040,40.787520,-73.986720,40.764965,2462,1932,552,2,1,23,0 days 00:11:09,2014-01-02 00:29:53
307786,2014-01-01 23:59:54,2014-01-02 00:07:32,-73.984210,40.760310,-73.999430,40.746100,1752,1315,287,2,1,23,0 days 00:07:38,2014-01-02 00:29:54
307787,2014-01-01 23:59:54,2014-01-02 00:16:15,-74.006000,40.735764,-73.965904,40.766144,1095,1908,708,2,1,23,0 days 00:16:21,2014-01-02 00:29:54
307788,2014-01-01 23:59:55,2014-01-02 00:06:56,-73.990425,40.730553,-73.988460,40.748577,922,1453,363,2,1,23,0 days 00:07:01,2014-01-02 00:29:55


In [47]:
%%time
# 获取可连接的需求 构建车辆共享网络
# 预知时长约束，当前需求至pred_time 内的出行
i = 302000
pred_data = data1[(data1['pickup_datetime'][i] <= data1['pickup_datetime']) & (data1['pickup_datetime'] <= data1['pred_time'][i])]
# 当前出行的终点
i_d = pred_data.iloc[0, :]['D']

# 空驶行程时间计算函数
def get_t_ij(j_o, i_d=i_d, G=G_ig):
    return G_ig.shortest_paths(i_d, j_o, weights='travel_time')[0][0]

# 当前出行终点前往pred_time 内的出行起点的行程时间 5.56s
t_ij = pred_data.iloc[1:, :]['D'].apply(get_t_ij).apply(get_timedelta)

# 当前出行终点的预计到达时刻
t_id = pred_data.iloc[0, :]['dropoff_datetime']

# 实际的延误时间
real_delay = (t_id + t_ij - pred_data.iloc[0:, :]['pickup_datetime'])
# 约束
# 当前出行终点+空驶行程时间需 小于等于 下一个行程的起点时间+允许的延误
condition1 = real_delay <= pd.Timedelta(minutes=delay)
# 确定行程时间 小于空驶阈值， 且无论其晚到达或者早到达，当前出行终点时刻与下一个行程的起点时刻应该相差在空驶阈值内
delta_two_trips = pred_data.iloc[1:, :]['pickup_datetime'] - t_id
condition2 = (t_ij <= pd.Timedelta(minutes=theta)) & (delta_two_trips <= pd.Timedelta(minutes=theta))
# 结合两个条件
condition = condition1 & condition2
# 当延误小于0时，没有乘客的时间等于delta_two_trips， 当延误大于等于0时，这个时间等于行驶时间
condition3 = (real_delay < pd.Timedelta(minutes=0))
pred_data['no_user_time'] = delta_two_trips[condition3].append(t_ij[~condition3]).sort_index()

# 共享网络边保存
share_trip = pred_data.iloc[1:, :][condition].index
save_data = pd.DataFrame({'D': share_trip})
save_data['O'] = i
save_data['no_user_time'] = pred_data['no_user_time'][share_trip].tolist()

print(len(save_data))

572
Wall time: 2.41 s
